In [10]:
import numpy as np
import pandas as pd

In [12]:
groups = ["instance", "terms"]
fields = ["lpBound New", "disjunctiveDualBound New", "lpBoundPostVpc New", "rootDualBound None", "rootDualBound New", "primalBound None", "Root Cuts (New)", "Root Cuts (None)"]

comp = pd.read_csv("/Users/sean/Documents/school/phd/research/warm_starting/vws/experiments/outputs/miplib_5000_gurobi/unperturbed_bound_table.csv")
comp

,instance,terms,lpBound New,disjunctiveDualBound New,lpBoundPostVpc New,rootDualBound None,rootDualBound New,primalBound None,Root Cuts (New),Root Cuts (None)
0,1,4,6637.188027,6641.585617,6640.550919,6714.074468,6716.532958,6742.200022,0.755580,0.732168
1,1,16,6637.188027,6644.289771,6642.855642,6714.074468,6716.497950,6742.200022,0.755246,0.732168
2,1,64,6637.188027,6647.648085,6641.628977,6714.074468,6708.438499,6742.200022,0.678498,0.732168
3,23588,4,7649.866134,7799.912563,7774.120220,7770.367871,7797.326487,8090.000000,0.335035,0.273784
4,23588,16,7649.866134,7867.860624,7862.267452,7770.367871,7867.450026,8090.000000,0.494358,0.273784
...,...,...,...,...,...,...,...,...,...,...
198,timtab1CUTS,16,618783.684884,624954.797154,620366.145397,632895.730479,633658.012677,764772.000000,0.101887,0.096666
199,timtab1CUTS,64,618783.684884,632296.230889,630413.602939,632895.730479,633482.153061,764772.000000,0.100682,0.096666
200,tr12-30,4,25302.209524,25611.359565,25498.481331,129146.721533,129090.840776,130596.000000,0.985705,0.986236
201,tr12-30,16,25302.209524,26546.641734,25805.491566,129146.721533,129090.840776,130596.000000,0.985705,0.986236


In [13]:
adf = pd.read_csv("aleks_results.csv")
adf = adf[["INSTANCE", "disj_terms", "LP OBJ", "BEST DISJ OBJ", "VPC OBJ", "AVG REF LAST_CUT_PASS", "AVG REF+V LAST_CUT_PASS", "ip_obj"]]
adf["rcnew"] = abs(adf["AVG REF+V LAST_CUT_PASS"] - adf["LP OBJ"]) / abs(adf["ip_obj"] - adf["LP OBJ"]) 
adf["rcnone"] = abs(adf["AVG REF LAST_CUT_PASS"] - adf["LP OBJ"]) / abs(adf["ip_obj"] - adf["LP OBJ"])
adf.columns = groups + fields
adf['instance'] = adf['instance'].str.replace('_presolved', '')
adf = adf.loc[(0 <= adf["Root Cuts (New)"]) & (adf["Root Cuts (New)"] <= 1)]

In [14]:
# join comp and adf on instance and terms
keep_cols = groups + ["Root Cuts (New)", "Root Cuts (None)"]
diff = comp[keep_cols].merge(adf[keep_cols], on=groups, suffixes=(" sean", " aleks"))
diff

,instance,terms,Root Cuts (New) sean,Root Cuts (None) sean,Root Cuts (New) aleks,Root Cuts (None) aleks
0,23588,4,0.335035,0.273784,0.318880,0.243813
1,23588,16,0.494358,0.273784,0.487854,0.243813
2,50v-10,4,0.745007,0.730938,0.744229,0.733697
3,50v-10,16,0.721864,0.730938,0.741268,0.733697
4,a1c1s1,4,0.890037,0.883432,0.887749,0.888510
...,...,...,...,...,...,...
176,timtab1CUTS,16,0.101887,0.096666,0.118558,0.113298
177,timtab1CUTS,64,0.100682,0.096666,0.113020,0.113298
178,tr12-30,4,0.985705,0.986236,0.983126,0.981713
179,tr12-30,16,0.985705,0.986236,0.983126,0.981713


In [15]:
diff[diff.columns[1:]].groupby("terms").mean()

,Root Cuts (New) sean,Root Cuts (None) sean,Root Cuts (New) aleks,Root Cuts (None) aleks
terms,,,,
4,0.565090,0.560543,0.565951,0.563010
16,0.568488,0.562422,0.574225,0.562029
64,0.567831,0.559230,0.576617,0.563988


In [16]:
diff[diff.columns[2:]].mean()

Root Cuts (New) sean      0.566773
Root Cuts (None) sean     0.560880
Root Cuts (New) aleks     0.570887
Root Cuts (None) aleks    0.562896
dtype: float64

In [17]:
adf[["terms", "Root Cuts (New)", "Root Cuts (None)"]].groupby("terms").mean()

,Root Cuts (New),Root Cuts (None)
terms,,
2.0,0.474030,0.465426
4.0,0.470454,0.460885
8.0,0.490194,0.467447
16.0,0.489893,0.455423
32.0,0.516469,0.462984
64.0,0.532016,0.468875
